In [9]:
from accounts import MasterAccount, Account, MultSigAccount
from transactions import Transaction, MultiSigTransaction
from os import urandom
#from database import WalletDB
from wallet_database_sqlite3 import Sqlite3Wallet
from blockcypher import get_address_details, get_addresses_details
from blockcypher import pushtx
from dotenv import load_dotenv
import os

import schedule 
import time 

In [62]:
class Wallet(MasterAccount):
    
    def __init__(self, depth, fingerprint, index, chain_code, private_key, db_user="neo4j",db_password="wallet", testnet = False):
    
        load_dotenv()
        BLOCKCYPHER_API_KEY = os.getenv('BLOCKCYPHER_API_KEY')
        
        
        #self.db = WalletDB( "neo4j://localhost:7687" ,db_user ,db_password )
        self.db = Sqlite3Wallet()
        super().__init__( depth, fingerprint, index, chain_code, private_key,testnet)
        
        #if not self.db.exist_wallet(self.get_xtended_key()):
            #self.db.new_wallet(self.get_xtended_key())
        
        #self.start_schedule()
        
    @classmethod
    def start_schedule(self):
        """
        This method starts the process of the scheduled tasks that are in charge of cleaning the database, 
        updating the balace automatically every hour, and update confirmations on new transactions sent or
        received.
        """
        schedule.every().day.do(self.clean_addresses)
        schedule.every().hour.do(self.update_balance)
        #pendind: to update confirmations of incoming and outcoming transactions.
        
    
    #@classmethod
    def clean_addresses(self):
        print("cleaning addresses...")
        self.db.clean_addresses()
        
    def close_db(self):
        self.db.close()
    
    @classmethod
    def get_i(self, index):
        if index is None:
            print(f"get i: {index}")
            i = int.from_bytes(urandom(4),"big")
            i = i & 0x7fffffff
            if i < (2**31-1):print("true")
        else:
            if index <= (2**31-1):
                i = index
            else:
                raise Exception (f"index must be less than {2**31-1} ")
        return i
    
    #@classmethod
    def get_unused_addresses_list(self, change_addresses=False, range_of_days=None, last_day_range=None):
        
        unused_addresses = self.db.get_unused_addresses(xprv = self.get_xtended_key(), 
                                                        days_range = range_of_days, 
                                                        max_days = last_day_range)
        
        if change_addresses:
            unused_addresses_filtered =  [x for x in unused_addresses if x["unused_address.type"]=="change"]
        else:
            unused_addresses_filtered =  [x for x in unused_addresses if x["unused_address.type"]=="recipient"]
        
        return unused_addresses_filtered
        
    #@classmethod
    def create_receiving_address(self, addr_type = "p2pkh",index=None):
        receiving_path = "m/0H/2H/"
        i = self.get_i(index)
        path = receiving_path + str(i)
        print(f"Path: {path}")
        receiving_xtended_acc = self.get_child_from_path(path)
        account = Account(int.from_bytes(receiving_xtended_acc.private_key,"big"),addr_type, self.testnet )
        self.db.new_address(account.address,i,False, self.get_xtended_key())
        return account

    #@classmethod
    def create_change_address(self,addr_type = "p2wpkh", index=None):
        change_path = "m/0H/1H/"
        i = self.get_i(index)
        path = change_path + str(i)
        print(f"Path: {path}")
        change_xtended_acc = self.get_child_from_path(path)
        account = Account(int.from_bytes(change_xtended_acc.private_key,"big"),addr_type, self.testnet )
        self.db.new_address(account.address,i,True, self.get_xtended_key())
        return account
    
    def get_utxos(self):
        return self.db.look_for_coins(self.get_xtended_key())
        
    def get_balance(self):
        coins = self.get_utxos()
        balance = 0
        for coin in coins:
            balance += coin["coin.amount"]
        return balance

    def update_balance(self):
        addresses_raw = self.db.get_all_addresses(self.get_xtended_key())
        addresses = [x["addr.address"] for x in addresses_raw]
        
        if self.testnet: coin_symbol = "btc-testnet"
        else: coin_symbol = "btc"
            
            
            
        for address in addresses_raw:
            
            #Blockcypher API call for address utxos
            addr_info = get_address_details(address["addr.address"], coin_symbol = coin_symbol, unspent_only=True)
            
            if addr_info["unconfirmed_n_tx"] > 0:
                for utxo in addr_info["unconfirmed_txrefs"]:
                    if not self.db.exist_utxo( utxo["tx_hash"], utxo["tx_output_n"], False):
                        print("new unconfirmed UTXO")
                        self.db.new_utxo(utxo["address"],utxo["tx_hash"],utxo["tx_output_n"],utxo["value"],False)
            
            if addr_info["n_tx"] - addr_info["unconfirmed_n_tx"] > 0 :
                for utxo in addr_info["txrefs"]:
                    if not self.db.exist_utxo( utxo["tx_hash"], utxo["tx_output_n"], True):
                        print("new confirmed UTXO")
                        self.db.new_utxo(addr_info["address"],utxo["tx_hash"],utxo["tx_output_n"],utxo["value"],True)
      
        return self.get_balance()
   

    def send(self, to_address_amount_list, segwit=True):
        """
        to_address_amount_list: List of touples. i.e. [(address1, amount1),(address2, amount2)...(addressn, amountn)]. 
        """
        total_amount = 0
        for output in to_address_amount_list:
            total_amount += output[1]
            
        balance = self.get_balance()
        if total_amount>balance:
            raise Exception(f"Not enough funds in wallet for this transaction.\nOnly {balance} satoshis available")
            
        all_utxos = self.get_utxos()
        utxos = []
        utxo_total = 0
        for utxo in all_utxos:
            if utxo["coin.amount"] > total_amount*1.1:
                utxos = [utxo]
                break
                
        if len(utxos)==0:    
            for utxo in all_utxos:
                utxos.append(utxo)
                utxo_total += utxo["coin.amount"]
                if utxo_total>total_amount: break
        change_account = self.create_change_address()
        
        if self.testnet: symbol = "btc-testnet"
        else: symbol= "btc"
          
        tx = Transaction.create_from_master( utxos,to_address_amount_list, self,change_account,
                           fee=None, segwit=segwit)
        
        push = pushtx(tx_hex= tx.transaction.serialize().hex(), coin_symbol=symbol,api_key=BLOCKCYPHER_API_KEY)
        print(f"TRANSACTION ID: {tx.transaction.id()}")
        
        #saving in db
        self.db.new_tx(tx.transaction.id(),[x["coin.local_index"] for x in utxos], 
                       #[str(x[0])+":"+str(x[1]) for x in to_address_amount_list]
                       [str(x) for x in tx.transaction.tx_outs]
                      )
        
        
        self.db.update_utxo(tx.transaction.id())
        
        return tx,push
        
    
    

In [63]:
words = "engine over neglect science fatigue dawn axis parent mind man escape era goose border invest slab relax bind desert hurry useless lonely frozen morning"
my_wallet = Wallet.recover_from_words(words, 256, "RobertPauslon",True)
my_wallet

8418572761394767459444418608530371567217744336876557229650655736270882421023962287566834689847418048023007033142570964655978481626857402470209587761075589


tprv8ZgxMBicQKsPfQJYjuFAso9x6STzmUdMh5U8CQqqQUTgtQHBHCq4C7FseeeZg15L16UeSwbrLwJRTXNPQsJQwqvbBA11sn4M6c3jR1LwAQP

In [10]:
unused_addresses = my_wallet.get_unused_addresses_list()
unused_addresses

[{'unused_address.address': 'muNHLmzC8CZ4nmZVy448xoYS84hmeXJXFG',
  'unused_address.acc_index': 1148426926,
  'unused_address.type': 'recipient'},
 {'unused_address.address': 'n1P8UFvu4WNGPiQBnRxNtBsEfC5KzU4QLb',
  'unused_address.acc_index': 198451525,
  'unused_address.type': 'recipient'}]

In [6]:
unused_receiving_addresses = [x for x in unused_addresses if x["unused_address.type"]=="recipient"]
unused_receiving_addresses

[{'unused_address.address': 'muNHLmzC8CZ4nmZVy448xoYS84hmeXJXFG',
  'unused_address.acc_index': 1148426926,
  'unused_address.type': 'recipient'},
 {'unused_address.address': 'n1P8UFvu4WNGPiQBnRxNtBsEfC5KzU4QLb',
  'unused_address.acc_index': 198451525,
  'unused_address.type': 'recipient'}]

In [54]:
my_wallet.update_balance()

new confirmed UTXO
<Record p=<Path start=<Node id=55 labels={'utxo'} properties={'transaction_id': '95339633e30b20336b586b1b569f5241b5cfd22a15c6636b20c184d91849d92d', 'out_index': 1, 'amount': 558441, 'address': 'tb1qpsp2zhty5m2s96rmcd45yhrn5cujddg7up93kw', 'spent': False, 'local_index': 16, 'confirmed': True}> end=<Node id=53 labels={'address'} properties={'address': 'tb1qpsp2zhty5m2s96rmcd45yhrn5cujddg7up93kw', 'type': 'change', 'acc_index': 996523316, 'created': 1590693482871}> size=1>>
new confirmed UTXO
<Record p=<Path start=<Node id=56 labels={'utxo'} properties={'transaction_id': 'f48f3e0517ea0adcb1344f807e7ea17c1a46f907e160ca0e49dfeeb102b3ee58', 'out_index': 1, 'amount': 438080, 'address': 'tb1qqxdd5svn80e2rtt3hgr8e8ufkacgwvyzjpsccw', 'spent': False, 'local_index': 17, 'confirmed': True}> end=<Node id=50 labels={'address'} properties={'address': 'tb1qqxdd5svn80e2rtt3hgr8e8ufkacgwvyzjpsccw', 'type': 'change', 'acc_index': 712926564, 'created': 1590685442931}> size=1>>


996521

In [66]:
my_wallet.get_balance()

991481

In [69]:
words = "client sudden sunset borrow pupil rely sand girl prefer movie bachelor guilt giraffe glove much strong dizzy switch ill silent goddess crumble goat power"
mywallet2 = Wallet.recover_from_words(words,entropy=256,passphrase="RobertPaulson",testnet=True)
mywallet2

6018716354834777059445516500463325153454936853151215624488541646839939367247233986759821493097378851875115552924255831805062990606698886137563075847103799


tprv8ZgxMBicQKsPdZiXgX96dFgvdso3ote8qGxbfb6KS961itG11jwLpAFEnhrpaKnqUn8XsjyEyvFEkhwSZfiqwJkhMgPDruDUh7hUTntjSHb

In [60]:
mywallet2.update_balance()

new confirmed UTXO
<Record p=<Path start=<Node id=59 labels={'utxo'} properties={'transaction_id': '95339633e30b20336b586b1b569f5241b5cfd22a15c6636b20c184d91849d92d', 'out_index': 0, 'amount': 999, 'address': 'mocc7fvLz4ggT5kVfLSJsiXfFByoXYBCqi', 'spent': False, 'local_index': 18, 'confirmed': True}> end=<Node id=25 labels={'address'} properties={'address': 'mocc7fvLz4ggT5kVfLSJsiXfFByoXYBCqi', 'type': 'recipient', 'acc_index': 2030716689, 'created': 1586490860048}> size=1>>
new confirmed UTXO
<Record p=<Path start=<Node id=60 labels={'utxo'} properties={'transaction_id': 'f48f3e0517ea0adcb1344f807e7ea17c1a46f907e160ca0e49dfeeb102b3ee58', 'out_index': 0, 'amount': 1700, 'address': 'mocc7fvLz4ggT5kVfLSJsiXfFByoXYBCqi', 'spent': False, 'local_index': 19, 'confirmed': True}> end=<Node id=25 labels={'address'} properties={'address': 'mocc7fvLz4ggT5kVfLSJsiXfFByoXYBCqi', 'type': 'recipient', 'acc_index': 2030716689, 'created': 1586490860048}> size=1>>


602699

In [12]:
mywallet2.create_receiving_address()

get i: None
true
Path: m/0H/2H/2030716689
<Record a=<Path start=<Node id=25 labels={'address'} properties={'address': 'mocc7fvLz4ggT5kVfLSJsiXfFByoXYBCqi', 'type': 'recipient', 'acc_index': 2030716689, 'created': 1586490860048}> end=<Node id=38 labels={'wallet'} properties={'xprv': 'tprv8ZgxMBicQKsPdZiXgX96dFgvdso3ote8qGxbfb6KS961itG11jwLpAFEnhrpaKnqUn8XsjyEyvFEkhwSZfiqwJkhMgPDruDUh7hUTntjSHb'}> size=1>>


Private Key Hex: ef14abc531a457eec55028301f5936e048f7921534828e985fcc8ca267c5d648

In [13]:
mywallet2.create_receiving_address()

get i: None
true
Path: m/0H/2H/1544325828
<Record a=<Path start=<Node id=12 labels={'address'} properties={'address': 'moY84CDzW5EmpKCK5GUg8VpqiKyUW1wbMA', 'type': 'recipient', 'acc_index': 1544325828, 'created': 1586485518978}> end=<Node id=38 labels={'wallet'} properties={'xprv': 'tprv8ZgxMBicQKsPdZiXgX96dFgvdso3ote8qGxbfb6KS961itG11jwLpAFEnhrpaKnqUn8XsjyEyvFEkhwSZfiqwJkhMgPDruDUh7hUTntjSHb'}> size=1>>


Private Key Hex: 3d3a25dcfdccfcbeb0de1f5f144e8b0b8340754502b9a1faeb5bfcb3bf89a1d8

In [65]:
my_tx = my_wallet.send([("muNHLmzC8CZ4nmZVy448xoYS84hmeXJXFG",200000)])

get i: None
true
Path: m/0H/1H/1755340894
<Record a=<Path start=<Node id=84 labels={'address'} properties={'address': 'tb1q2ufg6h5tax0tq7ehy6l9zwyyahy70yjj5fysq6', 'type': 'change', 'acc_index': 1755340894, 'created': 1590821620004}> end=<Node id=34 labels={'wallet'} properties={'xprv': 'tprv8ZgxMBicQKsPfQJYjuFAso9x6STzmUdMh5U8CQqqQUTgtQHBHCq4C7FseeeZg15L16UeSwbrLwJRTXNPQsJQwqvbBA11sn4M6c3jR1LwAQP'}> size=1>>
size of transaction without signatures: 240
fee: 5040
change 353401
total 558441, diff: 0
Address trying to spend from: tb1qpsp2zhty5m2s96rmcd45yhrn5cujddg7up93kw
tb1qpsp2zhty5m2s96rmcd45yhrn5cujddg7up93kw

ABOUT TO SIGN INPUT

SIGNING INPUT Segwit: 
Witness:
[b'0E\x02!\x00\xa3\xa8J4\x18\xbb;\xe2\xd3\xdd\xbc!\x83\x99\x14d\xad+\xac\x92I\xe3_\xce&\xd9\xdf3\xf3!}.\x02 d\x9e\xa9o\x03\x04\xa6\x04M\xb8\x93?c\xb9\xc9\xd8x\x1f\xca\x98\x07\x8a\xa6\x17\x0b\x19"\xd8\x8f\x92ee\x01', b'\x03\x89\x85Y\xa2h\xf85\xb2n@\xac\x87@iX\x9b\xfb\x138F\xb6\x0e\x82\x0e\x9a2\xf3\x06\xfe\x0e\xb3 '] 
script_pu

In [6]:
my_tx.transaction.serialize().hex()

'01000000000104231afd0f9f748fbbaf877b47d6fbcf9519b07d0423eb04e3e4732c0ab8476f5b000000006b483045022100a1f088a3f3f7cf22fbceb356b105cd9d6ffd9614fcaa7bf2cb69f4ab021c462c02205eccf7c6f02d81d101beceda4096484fc751d206dc5699720ddca648927c7c33012103a8a9d6b82d3ff26e50a730b47e2716eda4b4e553e4c34c42a57d11e1a69c816effffffff3b8088cbd2afba4be030454b36a45eb1117e1a4bcd073e8d426c96faaa5b5de4010000006b4830450221008683941381dac2da5be75c7d9c65063634af1528ffb9331ed1f73b52861e03820220281072000566a60753ebd5f0af832be73bb3825b44ce58e442a1d3e45ec87edc012102868ddcd03f6b246db94c0ef80ca349ad68c7176c3eeb8e1587eb3b7928b2e926ffffffff8b18481cbff7feb67b0e58451b9f7db1a0fc4ab91d31cca5563d927f72bfc884000000006a473044022048e71f0a4909822b274e8700f5e582fcd783189abb2963f4932465f6bade1dee02206a7611c5cd8a1eae68a6f7befecd70761f27a38676d69d74cd492e1368da12190121034c44a96e34cb349ee1a177a1458b2e19e42c16c1837f8895a4683995bc7eea4effffffffe54776d0f285d42f4a285f3afc320156ede1169a70c2ca59f5693928ce2df85a010000006a47304402207483537043b9894

In [5]:
my_tx2 = my_wallet.send([("mucD4dKRnaCzmFBeFh8GDYdJb4KRhEbyBk",200000)])

get i: None
true
Path: m/0H/1H/320043551
<Record a=<Path start=<Node id=37 labels={'address'} properties={'address': 'tb1qncg7k3tl77sq5ftedas8vm8dh0fvgde0d9j962', 'type': 'change', 'acc_index': 320043551, 'created': 1586487127667}> end=<Node id=34 labels={'wallet'} properties={'xprv': 'tprv8ZgxMBicQKsPfQJYjuFAso9x6STzmUdMh5U8CQqqQUTgtQHBHCq4C7FseeeZg15L16UeSwbrLwJRTXNPQsJQwqvbBA11sn4M6c3jR1LwAQP'}> size=1>>
size of transaction without signatures: 240
fee: 7200
change 649860
total 857060, diff: 0
Address trying to spend from: tb1qt3f852j7rudy56pwf3p5s5s8svcedn8lnkadnn
tb1qt3f852j7rudy56pwf3p5s5s8svcedn8lnkadnn

ABOUT TO SIGN INPUT

SIGNING INPUT Segwit: 
Witness:
[b'0E\x02!\x00\xd8\xb0\xb6\xfe\x922a\xec\xfe\xe6\x01\x11\xa37xtf\xef\x8aT\x9c\xf9\xbf\x12_\xfc\x82\xd0\xa3\xb4\xba\x9b\x02 \x16*\x95x9\x86\xc2\x8c\xf2FM\xe3\x922\x8c\xe3\xea\xc7\xa1i\xfb\xda\xdb\xd0\x14\xfd\xed\xb5\x98`\x82^\x01', b'\x03\x8b6\xf5O\xed\x01\xe2\x13\xe4:\x11V\xca\x97\xdf9\xe8#0@d\xa61@\xb0\xb3\x0f\xf0\x947D\x17'] 

In [6]:
my_tx2.transaction.serialize().hex()

'01000000000101d0b67263627c223533b658348cbe2e72be2ac64afa19c84ab74da14db1ed1c780100000000ffffffff02400d0300000000001976a9149a902b3d4f491183aa9def94453ccf6e91507a9388ac84ea0900000000001600149e11eb457ff7a00a25796f60766cedbbd2c4372f02483045022100d8b0b6fe923261ecfee60111a337787466ef8a549cf9bf125ffc82d0a3b4ba9b0220162a95783986c28cf2464de392328ce3eac7a169fbdadbd014fdedb59860825e0121038b36f54fed01e213e43a1156ca97df39e823304064a63140b0b30ff09437441700000000'

In [34]:
mywallet2.update_balance()

600000

In [29]:
my_tx = my_wallet.send([("mocc7fvLz4ggT5kVfLSJsiXfFByoXYBCqi",200000)])

get i: None
true
Path: m/0H/1H/1187776120
<Record a=<Path start=<Node id=29 labels={'address'} properties={'address': 'tb1qje5fsgd4gp866zak3frdxpyluf05zu3rvkg4j6', 'type': 'change', 'acc_index': 1187776120, 'created': 1586491589477}> end=<Node id=34 labels={'wallet'} properties={'xprv': 'tprv8ZgxMBicQKsPfQJYjuFAso9x6STzmUdMh5U8CQqqQUTgtQHBHCq4C7FseeeZg15L16UeSwbrLwJRTXNPQsJQwqvbBA11sn4M6c3jR1LwAQP'}> size=1>>
size of transaction without signatures: 240
fee: 5040
change 444820
total 649860, diff: 0
Address trying to spend from: tb1qncg7k3tl77sq5ftedas8vm8dh0fvgde0d9j962
tb1qncg7k3tl77sq5ftedas8vm8dh0fvgde0d9j962

ABOUT TO SIGN INPUT

SIGNING INPUT Segwit: 
Witness:
[b'0E\x02!\x00\x83\x87\x89|\xd8\xa8\xb3\xb5S\xdf`\xc6\nJgf\x1eq\n\xc5Wl\x9c\xfaA\xd8\x92\x8a\xbb\xf9\xdah\x02 ~\xdd/\x08\xef\x1a\x82y\xd2\x97"\x92\xe0_\x9dn\xf5\x9bN\xe2a\xcb:.F\xd9\xc4&o/<T\x01', b'\x02d\xc9\x00\xf76\x19\x84\xb8\xedR=\xd9S:\x03\x8b\xc7\xb8\x84\xd3)\x8awG\xf3\xfc\xc7\xe4\x98[\x1e\x95'] 
script_pubkey of tx IN

In [30]:
push = {'tx': {'block_height': -1, 'block_index': -1, 'hash': '73f1aa13e0d8b56c393f540e16fe395f3b7563a7022b9f299ecf7164dc33dc52', 'addresses': ['tb1qncg7k3tl77sq5ftedas8vm8dh0fvgde0d9j962', 'mocc7fvLz4ggT5kVfLSJsiXfFByoXYBCqi', 'tb1qje5fsgd4gp866zak3frdxpyluf05zu3rvkg4j6'], 'total': 644820, 'fees': 5040, 'size': 116, 'preference': 'medium', 'relayed_by': '45.32.199.25', 'received': '2020-04-10T04:06:32.550386564Z', 'ver': 1, 'double_spend': False, 'vin_sz': 1, 'vout_sz': 2, 'confirmations': 0, 'inputs': [{'prev_hash': '300d92fef1910d96dc414156ee73c8e878936fd0a303bf80804543cf5ac3cdee', 'output_index': 1, 'output_value': 649860, 'sequence': 4294967295, 'addresses': ['tb1qncg7k3tl77sq5ftedas8vm8dh0fvgde0d9j962'], 'script_type': 'pay-to-witness-pubkey-hash', 'age': 1696941, 'witness': ['30450221008387897cd8a8b3b553df60c60a4a67661e710ac5576c9cfa41d8928abbf9da6802207edd2f08ef1a8279d2972292e05f9d6ef59b4ee261cb3a2e46d9c4266f2f3c5401', '0264c900f7361984b8ed523dd9533a038bc7b884d3298a7747f3fcc7e4985b1e95']}], 'outputs': [{'value': 200000, 'script': '76a91458d2a84f2308ca6eb38c7f28f017b6afe210990588ac', 'addresses': ['mocc7fvLz4ggT5kVfLSJsiXfFByoXYBCqi'], 'script_type': 'pay-to-pubkey-hash'}, {'value': 444820, 'script': '001496689821b5404fad0bb68a46d3049fe25f417223', 'addresses': ['tb1qje5fsgd4gp866zak3frdxpyluf05zu3rvkg4j6'], 'script_type': 'pay-to-witness-pubkey-hash'}]}}

In [31]:
push

{'tx': {'block_height': -1,
  'block_index': -1,
  'hash': '73f1aa13e0d8b56c393f540e16fe395f3b7563a7022b9f299ecf7164dc33dc52',
  'addresses': ['tb1qncg7k3tl77sq5ftedas8vm8dh0fvgde0d9j962',
   'mocc7fvLz4ggT5kVfLSJsiXfFByoXYBCqi',
   'tb1qje5fsgd4gp866zak3frdxpyluf05zu3rvkg4j6'],
  'total': 644820,
  'fees': 5040,
  'size': 116,
  'preference': 'medium',
  'relayed_by': '45.32.199.25',
  'received': '2020-04-10T04:06:32.550386564Z',
  'ver': 1,
  'double_spend': False,
  'vin_sz': 1,
  'vout_sz': 2,
  'confirmations': 0,
  'inputs': [{'prev_hash': '300d92fef1910d96dc414156ee73c8e878936fd0a303bf80804543cf5ac3cdee',
    'output_index': 1,
    'output_value': 649860,
    'sequence': 4294967295,
    'addresses': ['tb1qncg7k3tl77sq5ftedas8vm8dh0fvgde0d9j962'],
    'script_type': 'pay-to-witness-pubkey-hash',
    'age': 1696941,
    'witness': ['30450221008387897cd8a8b3b553df60c60a4a67661e710ac5576c9cfa41d8928abbf9da6802207edd2f08ef1a8279d2972292e05f9d6ef59b4ee261cb3a2e46d9c4266f2f3c5401',
  

In [40]:
my_wallet.close_db()

In [8]:
import schedule 
import time 

In [41]:
load_dotenv()
BLOCKCYPHER_API_KEY = os.getenv('BLOCKCYPHER_API_KEY')

addr_info = get_addresses_details(["tb1qje5fsgd4gp866zak3frdxpyluf05zu3rvkg4j6","tb1qt3f852j7rudy56pwf3p5s5s8svcedn8lnkadnn",
                                  "mocc7fvLz4ggT5kVfLSJsiXfFByoXYBCqi","moY84CDzW5EmpKCK5GUg8VpqiKyUW1wbMA"], 
                                coin_symbol = "btc-testnet", unspent_only=True)

In [42]:
addr_info

[{'address': 'mocc7fvLz4ggT5kVfLSJsiXfFByoXYBCqi',
  'total_received': 202699,
  'total_sent': 0,
  'balance': 202699,
  'unconfirmed_balance': 0,
  'final_balance': 202699,
  'n_tx': 3,
  'unconfirmed_n_tx': 0,
  'final_n_tx': 3,
  'txrefs': [{'tx_hash': '95339633e30b20336b586b1b569f5241b5cfd22a15c6636b20c184d91849d92d',
    'block_height': 1746770,
    'tx_input_n': -1,
    'tx_output_n': 0,
    'value': 999,
    'ref_balance': 202699,
    'spent': False,
    'confirmations': 78,
    'confirmed': datetime.datetime(2020, 5, 28, 19, 35, 39, tzinfo=tzutc()),
    'double_spend': False},
   {'tx_hash': 'f48f3e0517ea0adcb1344f807e7ea17c1a46f907e160ca0e49dfeeb102b3ee58',
    'block_height': 1746763,
    'tx_input_n': -1,
    'tx_output_n': 0,
    'value': 1700,
    'ref_balance': 201700,
    'spent': False,
    'confirmations': 85,
    'confirmed': datetime.datetime(2020, 5, 28, 17, 15, 25, tzinfo=tzutc()),
    'double_spend': False},
   {'tx_hash': '73f1aa13e0d8b56c393f540e16fe395f3b7563a7

In [58]:
from urllib.request import Request, urlopen
import json
import os

def read_json(url):
    request = Request(url)
    response = urlopen(request)
    data = response.read()
    url2 = json.loads(data)
    return url2

api_key = os.getenv("CC_API")
url = f"https://min-api.cryptocompare.com/data/price?fsym=BTC&tsyms=USD"
raw_data = read_json(url)
raw_data["USD"]

9434.47

In [67]:
import pyperclip

In [68]:
pyperclip.copy("This is a copy test.")

In [2]:
from kivy_garden.zbarcam import ZBarCam

[INFO   ] [Logger      ] Record log in /Users/oscareduardosernarosero/.kivy/logs/kivy_20-06-01_30.txt
[INFO   ] [Kivy        ] v1.11.1
[INFO   ] [Kivy        ] Installed at "/Users/oscareduardosernarosero/anaconda3/lib/python3.7/site-packages/kivy/__init__.py"
[INFO   ] [Python      ] v3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
[INFO   ] [Python      ] Interpreter at "/Users/oscareduardosernarosero/anaconda3/bin/python"
[INFO   ] [Factory     ] 184 symbols loaded
[INFO   ] [Image       ] Providers: img_tex, img_imageio, img_dds, img_sdl2, img_pil, img_gif (img_ffpyplayer ignored)


In [22]:
txs = ["a:1","b:2","c:3"]
outs = [str(x).split(":")+[i] for i,x in enumerate(txs)]
outs

[['a', '1', 0], ['b', '2', 1], ['c', '3', 2]]